In [1]:
from generators.controller_runtime_shell import Controller, P4ProtoTxtParser
from generators.evaluation_parser import EvaluationPaser

import pandas as pd
import pickle
import numpy as np

from keras.models import load_model
from xgboost import XGBClassifier

2023-02-12 13:31:46.255019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 13:31:46.753099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-12 13:31:46.753204: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-12 13:31:48.883326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
############################################### setup ###############################################
p4_prog_name = "rf_new"
# p4_prog_name = "temp"

p4_info_path = f"../p4/build/{p4_prog_name}.p4info.txt"
p4_bin_path = f"../p4/build/{p4_prog_name}.json"
switch_grpc_addr = "127.0.0.1:50052"

# ml models paths
controller_models_dir = '../outputs/controller_ml/'

nn_model_dir = controller_models_dir + 'nn_model/'
rf_model_path = controller_models_dir + 'rf_model.pkl'
xgb_model_path = controller_models_dir + "xgb_model.json"

# setup the connection
my_controller = Controller(model_nn_dir=nn_model_dir, model_rf_path=rf_model_path, model_xgb_path=xgb_model_path)
my_controller.setUp(device_grpc_addr=switch_grpc_addr, 
                    device_id=0, 
                    p4_info_path=p4_info_path, 
                    p4_bin_path=p4_bin_path)

# parse the P4 proto text file
proto_parser = P4ProtoTxtParser(p4_info_path)

Loaded NN model.
Loaded RF model.
Loaded XGB model.
Setup the connection to switch.


CRITICAL:root:StreamChannel error, closing stream
CRITICAL:root:P4Runtime RPC error (UNAVAILABLE): Socket closed


In [11]:
############################################### shut down the connection ###############################################
my_controller.tearDown()

In [25]:
############################################### get features in training dataset ###############################################
# ATTENTION: The following features are not in the training dataset, but exists in P4 program
#       bidirectional_first_seen_ms
#       bidirectional_packets
#       splt_direction_1_0
#       splt_direction_1_2
#       splt_direction_2_0
#       splt_direction_3_0
#       splt_direction_4_0
#       splt_direction_5_0
#       splt_direction_6_0
#       splt_direction_7_0
#       splt_direction_8_0

dataset_path = "../dataset/balanced_wo_time/balanced_wo_time_equal_8/preprocessed_balanced_wo_time_equal_8_merged.csv"
df = pd.read_csv(dataset_path)
training_features = df.columns

In [3]:
############################################### process packetIn (part 1) ###############################################
# pathes
flow_length = 8
relevant_features_num = 20
feature_relevant_serialiazation_path = f"../outputs/serialization/balanced_wo_time/feature_relevant_balanced_wo_time_equal_{flow_length}_f_{relevant_features_num}_serialized.pkl"

# get header collection
pkt_in_header_id2name_dict = proto_parser.get_packet_in_id2name_dict()
pkt_in_header_name2id_dict = proto_parser.get_packet_in_name2id_dict()
pkt_out_header_list = proto_parser.get_packet_out_header()

# # load relevant features
# with open(feature_relevant_serialiazation_path, "rb") as f:
#     relevant_feature_list = pickle.load(f)

features_list = ['bidirectional_bytes', 'src2dst_packets', 'src2dst_bytes',
       'dst2src_packets', 'dst2src_bytes', 'bidirectional_min_ps',
       'bidirectional_mean_ps', 'bidirectional_max_ps', 'src2dst_min_ps',
       'src2dst_max_ps', 'dst2src_min_ps', 'dst2src_max_ps',
       'bidirectional_syn_packets', 'bidirectional_cwr_packets',
       'bidirectional_ece_packets', 'bidirectional_urg_packets',
       'bidirectional_ack_packets', 'bidirectional_psh_packets',
       'bidirectional_rst_packets', 'bidirectional_fin_packets',
       'src2dst_syn_packets', 'src2dst_cwr_packets', 'src2dst_ece_packets',
       'src2dst_urg_packets', 'src2dst_ack_packets', 'src2dst_psh_packets',
       'src2dst_rst_packets', 'src2dst_fin_packets', 'dst2src_syn_packets',
       'dst2src_cwr_packets', 'dst2src_ece_packets', 'dst2src_urg_packets',
       'dst2src_ack_packets', 'dst2src_psh_packets', 'dst2src_rst_packets',
       'dst2src_fin_packets', 'splt_ps_1', 'splt_ps_2', 'splt_ps_3',
       'splt_ps_4', 'splt_ps_5', 'splt_ps_6', 'splt_ps_7', 'splt_ps_8',
       'splt_direction_1_1', 'splt_direction_2_1', 'splt_direction_2_2',
       'splt_direction_3_1', 'splt_direction_3_2', 'splt_direction_4_1',
       'splt_direction_4_2', 'splt_direction_5_1', 'splt_direction_5_2',
       'splt_direction_6_1', 'splt_direction_6_2', 'splt_direction_7_1',
       'splt_direction_7_2', 'splt_direction_8_1', 'splt_direction_8_2']

In [ ]:
############################################### process packetIn (part 2) ###############################################
# sniff the packets from switch
pktIn_handler = my_controller.packetIn_handler
pktIn_handler.my_sniff(lambda pkt: my_controller.predict_flow(pkt, pkt_in_header_id2name_dict, features_list))

In [ ]:
############################################### get the average prediction time for NN, RF, XGB ###############################################
number_to_switch = my_controller.get_number_flow_to_switch()
avg_prediction_time_nn = my_controller.get_prediction_time_avg_nn()
avg_prediction_time_rf = my_controller.get_prediction_time_avg_rf()
avg_prediction_time_xgb = my_controller.get_prediction_time_avg_xgb()

print(f"number of flows classified in controller: {number_to_switch}")
print(f"average prediction time of NN: {avg_prediction_time_nn}")
print(f"average prediction time of RF: {avg_prediction_time_rf}")
print(f"average prediction time of XGB: {avg_prediction_time_xgb}")

In [14]:
my_controller.get_number_flow_to_switch()

0

In [5]:
################################################ save evaluation in csv  ################################################
# parameter setting
gini_threshold = 0.1
timeout_expiration = 1800 # second
evaluation_index = 1
evaluation_day_list = ["tuesday", "wednesday", "thursday", "friday"]
# evaluation_day_list = ["friday"]

# first columns dictionary in csv file
parameter_dict = {
    "evaluation_index": evaluation_index,
    "gini_threshold": gini_threshold,
    "timeout_expiration": timeout_expiration
}

# initialize paths
evaluation_file_path_list = []
csv_save_path_list = []
for day in evaluation_day_list:
    evaluation_file_path = f'../evaluation/{day}/{day}_evaluation_index_{evaluation_index}/{day}_txt_gini_{gini_threshold}_{evaluation_index}.txt'
    csv_save_path = f'../evaluation/{day}/{day}_csv.csv'
    evaluation_file_path_list.append(evaluation_file_path)
    csv_save_path_list.append(csv_save_path)

# save evaluation in csv file for each dataset
evalation_parser = EvaluationPaser()
for i, day in enumerate(evaluation_day_list):
    evalation_parser.parse_file(evaluation_file_path_list[i])
    evalation_parser.save_to_csv(csv_path=csv_save_path_list[i], keep_header=False, dataset_day=day, parameter_dict=parameter_dict)

In [ ]:
df = pd.read_csv(csv_save_path_list[3])
for column in df.columns:
    print(f"{column}: {df.iloc[0][column]}")

In [ ]:
import threading

def show():
    print("it's thread 1")

def show1():
    print("it's thread 2")


t1 = threading.Thread(target=show)
t2 = threading.Thread(target=show1)

t1.start()
t2.start()



## time_p4
- 0.004922 (3)
- 0.00889596875 (32)
- 0.00778157894736842105263157894737 (38)
- 0.010970875 (120)
- 0.20708834545454546 (495)
- 0.1078373282442748 (917)
- 0.09084598601102163 (4718)
- 0.23016635096735188 (6616)
- 0.2307465652053849 (14485)
- 0.18141805931414529 (26857)


## time_controller
- 0.6180241666666666 (18)
- 0.40646744117647058823529411764706 (34)
- 0.5851153153153152 (222)
- 0.6580463358778627 (262)
- 0.5068668896103896 (308)
- 0.4153190846063455 (851)
- 0.6655356756756756 (1591)
- 0.6661054379297865 (2763)
- 0.5803789624119029 (3831)